# HOLOCLEAN DEMO


In [1]:
from holofusion import HoloFusion, HoloFusionSession
from time import time as t

## Start Holoclean demo
Create HoloClean object and Session.
The HoloClean object will log all info for the test.
Session will be used to ingest data from input files.

In [2]:
        holo_obj = HoloFusion(majority_vote=0, training_data=1, multiple_weights=0)
        session = HoloFusionSession("Session", holo_obj)
        fx = open('execution_time.txt', 'w')
        

/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input from file

In [ ]:
        list_time = []
        start_time = t()

        session.ingest_dataset("data/clean_flight/flight-data-test.csv")
        d = t()-start_time
        list_time.append(d)
        fx.write('ingest csv time: '+str(d)+'\n')
        print 'ingest csv time: '+str(d)+'\n'
        
        start_time = t()
        session.adding_training_data("data/clean_flight/flight-data_training.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('adding training data: ' + str(d) + '\n')
        print 'adding training time: ' + str(d) + '\n'
        
        print 'Clean table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_clean", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()
        print 'Don\'t know table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_dk", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()

ingest csv time: 26.4400908947

['Source', 'Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
adding training time: 66.6959888935

Clean table
+--------------------+--------------------+--------------+-----------+
|              Source|          Flight_Num|Scheduled_Dept|Actual_Dept|
+--------------------+--------------------+--------------+-----------+
|                ifly|2011-12-01-AA-384...|          6:00|       6:56|
|                  aa|2011-12-01-AA-466...|          6:00|       6:08|
|             flights|2011-12-01-AA-466...|          6:00|       6:08|
| businesstravellogue|2011-12-01-AA-466...|          6:00|       6:08|
| businesstravellogue|2011-12-01-AA-205...|         10:40|      11:05|
|       flylouisville|2011-12-01-AA-205...|         10:40|      11:05|
|         flightstats|2011-12-01-AA-205...|         10:40|      11:05|
|           quicktrip|2011-12-01-AA-205...|         10:40|      11:05|
|world-flight-tracker|2011-12-01-AA-205..

## Featurization

In [ ]:
        start_time = t()
        session.feature()
        d = t() - start_time
        list_time.append(d)
        fx.write('creating feature table time: '+str(d)+'\n')
        print 'Feature table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Feature_temp", session.dataset)
        sql.show()
        print ' feature table time: '+str(d)+'\n'

['Source', 'Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
adding weight_id to feature table...
creating weight table
creating feature table with weights ids


## Inference


In [ ]:
        start_time = t()
        session.inference()
        d = t() - start_time
        list_time.append(d)
        fx.write('inference time: ' + str(d) + '\n')
        print 'Probabilities table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Probabilities", session.dataset)
        sql.show()
        print 'inference time: ' + str(d) + '\n'
        

## Final Table and Accuracy Calculations

In [ ]:
        start_time = t()

        session.accuracy("data/clean_flight/flight-data_truth.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('time to calculate accuracy: ' + str(d) + '\n')
        print 'time to calculate accuracy: ' + str(d) + '\n'
        fx.close()